# Part 1: Join: `dvd_rentals.rental` and `dvd_rentals.inventory`

## Purpose

In [1]:
-- Purpose 
SELECT * 
FROM dvd_rentals.rental
WHERE customer_id = 130
LIMIT 10; 

/* We need to keep all of the customer rental records from dvd_rentals.rental and match up
each record with its equivalent film_id value from the dvd_rentals.inventory table. */ 

(10 row(s) affected)

Total execution time: 00:00:00.105

rental_id,rental_date,inventory_id,customer_id,return_date,staff_id,last_update
1,2005-05-24T22:53:30,367,130,2005-05-26T22:04:30,1,2006-02-15T21:30:53
746,2005-05-29T09:25:10,4272,130,2005-06-02T04:20:10,2,2006-02-15T21:30:53
1630,2005-06-16T07:55:01,2413,130,2005-06-19T06:38:01,1,2006-02-15T21:30:53
1864,2005-06-17T01:39:47,1815,130,2005-06-24T19:39:47,2,2006-02-15T21:30:53
2163,2005-06-17T23:46:16,2600,130,2005-06-22T22:48:16,2,2006-02-15T21:30:53
2292,2005-06-18T07:37:48,173,130,2005-06-20T02:45:48,2,2006-02-15T21:30:53
2535,2005-06-19T01:39:04,901,130,2005-06-28T01:33:04,2,2006-02-15T21:30:53
2982,2005-06-20T08:38:29,2574,130,2005-06-28T13:21:29,1,2006-02-15T21:30:53
4339,2005-07-07T18:41:42,3215,130,2005-07-08T13:00:42,1,2006-02-15T21:30:53
4485,2005-07-08T01:07:54,2614,130,2005-07-16T03:19:54,2,2006-02-15T21:30:53


## Hypotheses

In [1]:
/* Hypothesis 1: no. of unique inventory_id records would be equal 
in both `rental` and `inventory` tables */
SELECT 
    COUNT(DISTINCT inventory_id)
FROM dvd_rentals.rental; 


SELECT
    COUNT(DISTINCT inventory_id)
FROM dvd_rentals.inventory; 


-- Rejected 
/* 1 additional inventory_id value in the dvd_rentals.inventory table 
compared to the dvd_rentals.rental table */

(1 row(s) affected)

(1 row(s) affected)

Total execution time: 00:00:00.039

count
4580


count
4581


In [2]:
-- 
/* H2 There will be a multiple records per unique inventory_id in the 
dvd_rentals.rental table */
SELECT
    inventory_id as target_column_values, 
    COUNT(*) as record_counts
FROM dvd_rentals.rental
GROUP BY target_column_values
ORDER BY record_counts ASC
LIMIT 10

/* Not much insight: we can only see (a) max number of times a unique inventory_id appears and/or (b) the top ranking 
inventory_id values */

(10 row(s) affected)

Total execution time: 00:00:00.096

target_column_values,record_counts
2786,1
1580,1
2662,1
3372,1
2172,2
3097,2
4519,2
2784,2
4221,2
663,2


In [3]:
/* H2: there will be multiple records per unique inventory_id in 
dvd_rentals.rental */

-- first generate group by counts on the target_column_values column
WITH counts_base AS (
SELECT
    inventory_id as target_column_values, 
    COUNT(*) as record_counts
FROM dvd_rentals.rental
GROUP BY target_column_values
)

/* summarize the group by counts above by grouping again on the row_counts 
from counts_base CTE part */
SELECT 
    record_counts, 
    COUNT(target_column_values) AS count_of_target_values
FROM counts_base
GROUP BY record_counts
ORDER BY record_counts; 


(5 row(s) affected)

Total execution time: 00:00:00.065

record_counts,count_of_target_values
1,4
2,1126
3,1151
4,1160
5,1139


In [4]:
/* H3: There will be multiple inventory_id records per unique film_id value 
in the dvd_rentals.inventory table */ 

-- first generate group by counts on the target_column_values column

WITH counts_base AS (
SELECT
    film_id as target_column_values,
    COUNT(DISTINCT inventory_id) as unique_record_counts
FROM dvd_rentals.inventory
GROUP BY target_column_values
)

/* summarize the group by counts above by grouping again on the row_counts 
from counts_base CTE part */

SELECT 
    unique_record_counts, 
    COUNT(target_column_values) AS count_of_target_values
FROM counts_base
GROUP BY unique_record_counts
ORDER BY unique_record_counts; 

-- Hypothesis Confirmed 

(7 row(s) affected)

Total execution time: 00:00:00.045

unique_record_counts,count_of_target_values
2,133
3,131
4,183
5,136
6,187
7,116
8,72


## Returning to the 2 Key Questions

### 1. How many records exist per `inventory_id` in `rental` or `inventory` table

`rental` distribution analysis on `inventory_id` foreign key

In [5]:
WITH counts_base AS(
SELECT
    inventory_id AS foreign_key_values,
    COUNT(*) AS row_counts
FROM dvd_rentals.rental
GROUP BY foreign_key_values
)

SELECT
    row_counts,
    COUNT(foreign_key_values) AS count_of_foreign_keys
FROM counts_base
GROUP BY row_counts
ORDER BY row_counts; 

-- 1-to-many relationship
/* Interpretation: 4 of our foreign key values (the inventory_id in this case) 
will have only a single row in the rental_table */

(5 row(s) affected)

Total execution time: 00:00:00.047

row_counts,count_of_foreign_keys
1,4
2,1126
3,1151
4,1160
5,1139


<span style="color: #800000;">`inventory`</span> distribution analysis on <span style="color: #800000;">`inventory_id`</span> foreign key

In [6]:
WITH counts_base AS(
SELECT
    inventory_id AS foreign_key_values,
    COUNT(*) AS row_counts
FROM dvd_rentals.inventory
GROUP BY foreign_key_values
)

SELECT
    row_counts,
    COUNT(foreign_key_values) AS count_of_foreign_keys
FROM counts_base
GROUP BY row_counts
ORDER BY row_counts; 

-- 1-to-1 relationship

-- Confirmation: Order by descending order to see the biggest record_count values 
SELECT 
    inventory_id, 
    COUNT(*) as record_count
FROM dvd_rentals.inventory
GROUP BY inventory_id
ORDER BY record_count DESC
LIMIT 5; 
-- 1-to-1 relationship

(1 row(s) affected)

(5 row(s) affected)

Total execution time: 00:00:00.056

row_counts,count_of_foreign_keys
1,4581


inventory_id,record_count
273,1
3936,1
2574,1
951,1
1489,1


### 2. How many overlapping and missing unique `foreign key` values are there between two tables?

In [7]:
-- how many foreign keys only exist in the left table and not in the right?
SELECT 
    COUNT(DISTINCT rental.inventory_id)
FROM dvd_rentals.rental
WHERE NOT EXISTS (
    SELECT inventory_id
    FROM dvd_rentals.inventory
    WHERE rental.inventory_id = inventory.inventory_id
)

(1 row(s) affected)

Total execution time: 00:00:00.067

count
0


In [8]:
-- how many foreign keys only exist in the right table and not in the left?

SELECT 
    COUNT(DISTINCT inventory.inventory_id)
FROM dvd_rentals.inventory
WHERE NOT EXISTS (
    SELECT inventory_id
    FROM dvd_rentals.rental
    WHERE rental.inventory_id = inventory.inventory_id
)

(1 row(s) affected)

Total execution time: 00:00:00.047

count
1


In [9]:
-- Inspection of FKV that only exists in the right table 
SELECT *
FROM dvd_rentals.inventory
WHERE NOT EXISTS (
    SELECT inventory_id
    FROM dvd_rentals.rental
    WHERE rental.inventory_id = inventory.inventory_id
)

/* It's possible that this particular rental inventory unit 
was never rented out by a customer */

(1 row(s) affected)

Total execution time: 00:00:00.097

inventory_id,film_id,store_id,last_update
5,1,2,2006-02-15T05:09:17


In [10]:
-- Joint overlap analysis 
SELECT 
    COUNT(DISTINCT rental.inventory_id)
FROM dvd_rentals.rental
WHERE EXISTS (
    SELECT inventory_id
    FROM dvd_rentals.inventory
    WHERE rental.inventory_id = inventory.inventory_id
)

(1 row(s) affected)

Total execution time: 00:00:00.048

count
4580


## Implementing Joins 

Inspecting whether number of `raw_record_counts` and `unique_foreign_key` values are equal in outputs resulting from both types of join

In [12]:
DROP TABLE IF EXISTS left_rental_join; 
CREATE TEMP TABLE left_rental_join AS 
SELECT
    rental.customer_id,
    rental.inventory_id, 
    inventory.film_id
FROM dvd_rentals.rental
LEFT JOIN dvd_rentals.inventory
    ON rental.inventory_id = inventory.inventory_id; 

DROP TABLE IF EXISTS inner_rental_join; 
CREATE TEMP TABLE inner_rental_join AS 
SELECT
    rental.customer_id,
    rental.inventory_id, 
    inventory.film_id
FROM dvd_rentals.rental
INNER JOIN dvd_rentals.inventory
    ON rental.inventory_id = inventory.inventory_id; 

-- check counts for each output 
-- parantheses used to make code look cleaner
    
(
    SELECT 
        'left join' AS join_type,
        COUNT(*) AS record_count, 
        COUNT(DISTINCT inventory_id) AS unique_key_values
    FROM left_rental_join
)
UNION
(
    SELECT 
        'inner join' AS join_type,
        COUNT(*) AS record_count, 
        COUNT(DISTINCT inventory_id) AS unique_key_values
    FROM inner_rental_join
)

-- no difference between an inner join or left join for our datasets

Commands completed successfully

Commands completed successfully

Commands completed successfully

Commands completed successfully

(2 row(s) affected)

Total execution time: 00:00:00.380

join_type,record_count,unique_key_values
inner join,16044,4580
left join,16044,4580


# Part 2: Joining `dvd_rentals.inventory` and `dvd_rentals.film_id`

We are going to follow the same joining journey as above and therefore, some steps have been skipped.   
1. Defining Purpose  
2. Coming up with contextual hypotheses about the data and validating them  
3. Distribution of foreign keys within each table  
4. No. of unique foreign key values that exist in each table

In [13]:
/* H1: 1-to-many relationship for film_id and rows 
of dvd_rentals.inventory table */

WITH counts_base AS ( 
SELECT 
    film_id as film_id_values, 
    COUNT(*) AS row_count
FROM dvd_rentals.inventory
GROUP BY film_id
)

SELECT
    row_count, 
    COUNT(DISTINCT film_id_values) AS unique_film_id_values
FROM counts_base
GROUP BY row_count
ORDER BY row_count

-- Hypothesis validated

(7 row(s) affected)

Total execution time: 00:00:00.035

row_count,unique_film_id_values
2,133
3,131
4,183
5,136
6,187
7,116
8,72


In [14]:
-- H2: 1-to-1 relationship for film_id and the rows of the dvd_rentals.film

SELECT
    film_id, 
    COUNT(*) AS record_count
FROM dvd_rentals.film
GROUP BY film_id
ORDER BY record_count DESC
LIMIT 5; 

-- Retrieving last 5 rows in descending order to confirm hypothesis
-- Hypothesis validated


(5 row(s) affected)

Total execution time: 00:00:00.166

film_id,record_count
273,1
51,1
951,1
839,1
652,1


In [15]:
-- Overlap and unique-value analysis 
-- FKVs only in left and not the right table
SELECT
    COUNT(DISTINCT inventory.film_id)
FROM dvd_rentals.inventory
WHERE NOT EXISTS (
    SELECT 
        film_id
    FROM dvd_rentals.film
    WHERE inventory.film_id = film.film_id
)

-- All unique film_id values that exist in `inventory` also exist in `film` 

(1 row(s) affected)

Total execution time: 00:00:00.035

count
0


In [16]:
-- only in right 
SELECT
    COUNT(DISTINCT film.film_id)
FROM dvd_rentals.film
WHERE NOT EXISTS (
    SELECT 
        film_id
    FROM dvd_rentals.inventory
    WHERE inventory.film_id = film.film_id
)

-- 42 unique film_id values that exist in `film` do not exist in `inventory`

(1 row(s) affected)

Total execution time: 00:00:00.031

count
42


In [17]:
-- Foreign Key values that exist only in right table
SELECT *
FROM dvd_rentals.film
WHERE NOT EXISTS (
    SELECT 
        film_id
    FROM dvd_rentals.inventory
    WHERE inventory.film_id = film.film_id
)
LIMIT 5; 
-- 42 films that do not have any inventories?

(5 row(s) affected)

Total execution time: 00:00:00.054

film_id,title,description,release_year,language_id,original_language_id,rental_duration,rental_rate,length,replacement_cost,rating,last_update,special_features,fulltext
14,ALICE FANTASIA,A Emotional Drama of a A Shark And a Database Administrator who must Vanquish a Pioneer in Soviet Georgia,2006,1,NULL,6,0.99,94,23.99,NC-17,2006-02-15T05:03:42,"[""Trailers"", ""Deleted Scenes"", ""Behind the Scenes""]",'administr':13 'alic':1 'databas':12 'drama':5 'emot':4 'fantasia':2 'georgia':21 'must':15 'pioneer':18 'shark':9 'soviet':20 'vanquish':16
33,APOLLO TEEN,A Action-Packed Reflection of a Crocodile And a Explorer who must Find a Sumo Wrestler in An Abandoned Mine Shaft,2006,1,NULL,5,2.99,153,15.99,PG-13,2006-02-15T05:03:42,"[""Trailers"", ""Commentaries"", ""Deleted Scenes"", ""Behind the Scenes""]",'abandon':22 'action':5 'action-pack':4 'apollo':1 'crocodil':10 'explor':13 'find':16 'mine':23 'must':15 'pack':6 'reflect':7 'shaft':24 'sumo':18 'teen':2 'wrestler':19
36,ARGONAUTS TOWN,A Emotional Epistle of a Forensic Psychologist And a Butler who must Challenge a Waitress in An Abandoned Mine Shaft,2006,1,NULL,7,0.99,127,12.99,PG-13,2006-02-15T05:03:42,"[""Trailers"", ""Commentaries""]",'abandon':20 'argonaut':1 'butler':12 'challeng':15 'emot':4 'epistl':5 'forens':8 'mine':21 'must':14 'psychologist':9 'shaft':22 'town':2 'waitress':17
38,ARK RIDGEMONT,A Beautiful Yarn of a Pioneer And a Monkey who must Pursue a Explorer in The Sahara Desert,2006,1,NULL,6,0.99,68,25.99,NC-17,2006-02-15T05:03:42,"[""Trailers"", ""Commentaries"", ""Deleted Scenes"", ""Behind the Scenes""]",'ark':1 'beauti':4 'desert':20 'explor':16 'monkey':11 'must':13 'pioneer':8 'pursu':14 'ridgemont':2 'sahara':19 'yarn':5
41,ARSENIC INDEPENDENCE,A Fanciful Documentary of a Mad Cow And a Womanizer who must Find a Dentist in Berlin,2006,1,NULL,4,0.99,137,17.99,PG,2006-02-15T05:03:42,"[""Trailers"", ""Deleted Scenes"", ""Behind the Scenes""]",'arsenic':1 'berlin':19 'cow':9 'dentist':17 'documentari':5 'fanci':4 'find':15 'independ':2 'mad':8 'must':14 'woman':12


In [18]:
-- overlapping foreign key values between `inventory` and `film` tables 
-- overlap
SELECT 
    COUNT (DISTINCT inventory.film_id)
FROM dvd_rentals.inventory
WHERE EXISTS (
    SELECT film_id
    FROM dvd_rentals.film
    WHERE inventory.film_id = film.film_id
)


(1 row(s) affected)

Total execution time: 00:00:00.034

count
958


In [19]:
-- Join Implementation & Output count comparisons: `Inner` vs. `Left` 
DROP TABLE IF EXISTS inner_join_pt_2; 
CREATE TEMP TABLE inner_join_pt_2 AS 
SELECT
    inventory.inventory_id, 
    inventory.film_id, 
    film.title
FROM dvd_rentals.inventory
INNER JOIN dvd_rentals.film
    ON inventory.film_id = film.film_id; 

DROP TABLE IF EXISTS left_join_pt_2; 
CREATE TEMP TABLE left_join_pt_2 AS 
SELECT
    inventory.inventory_id, 
    inventory.film_id, 
    film.title
FROM dvd_rentals.inventory
LEFT JOIN dvd_rentals.film
    ON inventory.film_id = film.film_id; 


(
    SELECT
        'inner join' AS join_type,
        COUNT(*) as record_count, 
        COUNT(DISTINCT film_id) AS unique_inventory_ids
    FROM inner_join_pt_2
)
UNION
(
    SELECT
        'left join' AS join_type,
        COUNT(*) as record_count, 
        COUNT(DISTINCT film_id) AS unique_inventory_ids
    FROM left_join_pt_2
)

--  no difference between left or inner joins 

NOTICE: table "inner_join_pt_2" does not exist, skipping

Commands completed successfully

Commands completed successfully

NOTICE: table "left_join_pt_2" does not exist, skipping

Commands completed successfully

Commands completed successfully

(2 row(s) affected)

Total execution time: 00:00:00.050

join_type,record_count,unique_inventory_ids
inner join,4581,958
left join,4581,958


# Joining Parts 1 and 2 

In [20]:
-- Creating a TEMP TABLE for the join

DROP TABLE IF EXISTS join_parts_1_and_2; 
CREATE TEMP TABLE join_parts_1_and_2 AS
SELECT
    rental.customer_id, 
    inventory.film_id, 
    film.title
FROM dvd_rentals.rental
INNER JOIN dvd_rentals.inventory
    ON rental.inventory_id = inventory.inventory_id
INNER JOIN dvd_rentals.film
    ON inventory.film_id = film.film_id; 


SELECT *
FROM join_parts_1_and_2
LIMIT 10; 


NOTICE: table "join_parts_1_and_2" does not exist, skipping

Commands completed successfully

Commands completed successfully

(10 row(s) affected)

Total execution time: 00:00:00.071

customer_id,film_id,title
130,80,BLANKET BEVERLY
459,333,FREAKY POCUS
408,373,GRADUATE LORD
333,535,LOVE SUICIDES
222,450,IDOLS SNATCHERS
549,613,MYSTIC TRUMAN
269,870,SWARM GOLD
239,510,LAWLESS VISION
126,565,MATRIX SNOWMAN
399,396,HANGING DEEP


# Parts 3: Joining `film_id` with `film_category`

In [21]:
-- `film_id` distribution on `dvd_rentals.film` 1-to-1 (already explored)

-- `film_id` distribution on `dvd_rentals.film_category` 1-to-1 
/* we would expect a unique film_id to only have one category, 
but we would expect a unique category_id to have multiple rows in 
film_category */

SELECT 
    film_id, 
    COUNT(*) AS row_count
FROM dvd_rentals.film_category
GROUP BY film_id
ORDER BY row_count DESC 
LIMIT 5; 

WITH base_counts AS (
SELECT 
    category_id, 
    COUNT(*) AS row_count
FROM dvd_rentals.film_category
GROUP BY category_id
)

SELECT 
    row_count, 
    COUNT(DISTINCT category_id) AS no_of_unique_category_ids
FROM base_counts
GROUP BY row_count
ORDER BY row_count DESC; 



(5 row(s) affected)

(14 row(s) affected)

Total execution time: 00:00:00.099

film_id,row_count
273,1
51,1
951,1
839,1
652,1


row_count,no_of_unique_category_ids
74,1
73,1
69,1
68,1
66,1
64,1
63,1
62,1
61,2
60,1


In [22]:
/* Foreign Key values that only exist in `dvd_rentals.film` and 
not in `dvd_rentals.film_category` */
SELECT
    COUNT(DISTINCT film.film_id)
FROM dvd_rentals.film
WHERE NOT EXISTS (
    SELECT film_id
    FROM dvd_rentals.film_category
    WHERE film.film_id = film_category.film_id
); 

/* Foreign Key values that only exist in `dvd_rentals.film_category` and 
not in `dvd_rentals.film` */
SELECT
    COUNT(DISTINCT film_category.film_id)
FROM dvd_rentals.film_category
WHERE NOT EXISTS (
    SELECT film_id
    FROM dvd_rentals.film
    WHERE film.film_id = film_category.film_id
)

(1 row(s) affected)

(1 row(s) affected)

Total execution time: 00:00:00.058

count
0


count
0


In [23]:
-- count of foriegn key values that exist in both tables
SELECT
    COUNT(DISTINCT film.film_id)
FROM dvd_rentals.film
WHERE EXISTS (
    SELECT film_id
    FROM dvd_rentals.film_category
    WHERE film.film_id = film_category.film_id
); 

(1 row(s) affected)

Total execution time: 00:00:00.012

count
1000


In [24]:
-- Join Implementation & Output count comparisons: `Inner` vs. `Left`
DROP TABLE IF EXISTS inner_join_pt_3; 

CREATE TEMP TABLE inner_join_pt_3 AS 
SELECT 
    film.film_id, 
    film_category.category_id
FROM dvd_rentals.film
INNER JOIN dvd_rentals.film_category
    ON film.film_id = film_category.film_id; 


DROP TABLE IF EXISTS left_join_pt_3; 

CREATE TEMP TABLE left_join_pt_3 AS 
SELECT 
    film.film_id, 
    film_category.category_id
FROM dvd_rentals.film
LEFT JOIN dvd_rentals.film_category
    ON film.film_id = film_category.film_id; 

(
    SELECT
        'inner join' AS join_type,
        COUNT(*) AS row_record_count, 
        COUNT(DISTINCT film_id) AS unique_foreign_value_count
    FROM inner_join_pt_3
)
UNION

(
    SELECT
        'left join' AS join_type,
        COUNT(*) AS row_record_count, 
        COUNT(DISTINCT film_id) AS unique_foreign_value_count
    FROM left_join_pt_3
)

-- No different in raw record counts and unique foreign key value counts

NOTICE: table "inner_join_pt_3" does not exist, skipping

Commands completed successfully

Commands completed successfully

NOTICE: table "left_join_pt_3" does not exist, skipping

Commands completed successfully

Commands completed successfully

(2 row(s) affected)

Total execution time: 00:00:00.072

join_type,row_record_count,unique_foreign_value_count
inner join,1000,1000
left join,1000,1000


# Part 4: Joining `film_category` with `category`

In [25]:
WITH base_counts AS (
SELECT 
    category_id, 
    COUNT(*) AS row_count
FROM dvd_rentals.film_category
GROUP BY category_id
)

SELECT 
    row_count, 
    COUNT(DISTINCT category_id) AS count_of_unique_cat_id
FROM base_counts
GROUP BY row_count
ORDER BY row_count 

/* 1-to-many relationship for `category_id` and rows of 
dvd_rentals.film_category table */

(14 row(s) affected)

Total execution time: 00:00:00.021

row_count,count_of_unique_cat_id
51,1
56,1
57,2
58,1
60,1
61,2
62,1
63,1
64,1
66,1


In [26]:
-- 1-to-1 
SELECT 
    category_id, 
    COUNT(*) AS record_count
FROM dvd_rentals.category
GROUP BY category_id
ORDER BY record_count DESC 
LIMIT 5; 
/* 1-to-1 relationship for `category_id` and rows of 
dvd_rentals.category table */

(5 row(s) affected)

Total execution time: 00:00:00.032

category_id,record_count
10,1
6,1
13,1
2,1
4,1


In [27]:
-- no. of unique FKVs in each table
-- unique category_id values in the left table and not the right table 

-- in left, but not right 
SELECT 
    COUNT(DISTINCT film_category.category_id)
FROM dvd_rentals.film_category
WHERE NOT EXISTS ( 
    SELECT 
        category_id
    FROM dvd_rentals.category
    WHERE film_category.category_id = category.category_id
); 

-- in right, but not left 
SELECT 
    COUNT(DISTINCT category.category_id)
FROM dvd_rentals.category
WHERE NOT EXISTS ( 
    SELECT 
        category_id
    FROM dvd_rentals.film_category
    WHERE film_category.category_id = category.category_id
)

(1 row(s) affected)

(1 row(s) affected)

Total execution time: 00:00:00.033

count
0


count
0


In [28]:
-- # FKVs in both tables
SELECT 
    COUNT(DISTINCT film_category.category_id)
FROM dvd_rentals.film_category
WHERE EXISTS ( 
    SELECT 
        category_id
    FROM dvd_rentals.category
    WHERE film_category.category_id = category.category_id
)

(1 row(s) affected)

Total execution time: 00:00:00.024

count
16


In [29]:
DROP TABLE IF EXISTS inner_join_pt_4; 

CREATE TEMP TABLE inner_join_pt_4 AS 
SELECT 
    film_category.film_id,
    film_category.category_id, 
    category.name
FROM dvd_rentals.film_category
INNER JOIN dvd_rentals.category
    ON film_category.category_id = category.category_id; 


DROP TABLE IF EXISTS left_join_pt_4; 

CREATE TEMP TABLE left_join_pt_4 AS 
SELECT 
    film_category.film_id,
    film_category.category_id, 
    category.name
FROM dvd_rentals.film_category
LEFT JOIN dvd_rentals.category
    ON film_category.category_id = category.category_id; 

(
    SELECT
        'inner join' AS join_type,
        COUNT(*) AS row_record_count, 
        COUNT(DISTINCT category_id) AS unique_category_ids
    FROM inner_join_pt_4
)
UNION

(
    SELECT
        'left join' AS join_type,
        COUNT(*) AS row_record_count, 
        COUNT(DISTINCT category_id) AS unique_category_ids
    FROM left_join_pt_4
)
-- no difference between `inner` and `left` joins

NOTICE: table "inner_join_pt_4" does not exist, skipping

Commands completed successfully

Commands completed successfully

NOTICE: table "left_join_pt_4" does not exist, skipping

Commands completed successfully

Commands completed successfully

(2 row(s) affected)

Total execution time: 00:00:00.087

join_type,row_record_count,unique_category_ids
inner join,1000,16
left join,1000,16


# Joining Multiple Tables (1, 2, 3, 4)

In [30]:
-- Joining parts 1, 2, 3, 4 using multiple table joins 
DROP TABLE IF EXISTS complete_join_dataset; 
CREATE TEMP TABLE complete_join_dataset AS 
SELECT
    rental.customer_id, 
    inventory.film_id, 
    film.title, 
    film_category.category_id, 
    category.name AS category_name 
FROM dvd_rentals.rental
INNER JOIN dvd_rentals.inventory
    ON rental.inventory_id = inventory.inventory_id
INNER JOIN dvd_rentals.film
    ON inventory.film_id = film.film_id 
INNER JOIN dvd_rentals.film_category
    ON film.film_id = film_category.film_id 
INNER JOIN dvd_rentals.category
    ON film_category.category_id = category.category_id; 

SELECT *
FROM complete_join_dataset
LIMIT 5; 

NOTICE: table "complete_join_dataset" does not exist, skipping

Commands completed successfully

Commands completed successfully

(5 row(s) affected)

Total execution time: 00:00:00.092

customer_id,film_id,title,category_id,category_name
130,80,BLANKET BEVERLY,8,Family
459,333,FREAKY POCUS,12,Music
408,373,GRADUATE LORD,3,Children
333,535,LOVE SUICIDES,11,Horror
222,450,IDOLS SNATCHERS,3,Children


In [31]:
/* Confirming that there's no data loss when using an `inner` join 
instead of a `left` join */

DROP TABLE IF EXISTS complete_joint_dataset; 

CREATE TEMP TABLE complete_joint_dataset AS 
SELECT
    rental.customer_id, 
    inventory.film_id, 
    film.title, 
    film_category.category_id, 
    category.name AS category_name 
FROM dvd_rentals.rental
INNER JOIN dvd_rentals.inventory
    ON rental.inventory_id = inventory.inventory_id
INNER JOIN dvd_rentals.film
    ON inventory.film_id = film.film_id 
INNER JOIN dvd_rentals.film_category
    ON film.film_id = film_category.film_id 
INNER JOIN dvd_rentals.category
    ON film_category.category_id = category.category_id; 


DROP TABLE IF EXISTS complete_joint_dataset_left; 

CREATE TEMP TABLE complete_joint_dataset_left AS 
SELECT
    rental.customer_id, 
    inventory.film_id, 
    film.title, 
    film_category.category_id, 
    category.name AS category_name 
FROM dvd_rentals.rental
LEFT JOIN dvd_rentals.inventory
    ON rental.inventory_id = inventory.inventory_id
LEFT JOIN dvd_rentals.film
    ON inventory.film_id = film.film_id 
LEFT JOIN dvd_rentals.film_category
    ON film.film_id = film_category.film_id 
LEFT JOIN dvd_rentals.category
    ON film_category.category_id = category.category_id; 

SELECT 
    'inner join' AS join_type,
    COUNT(*) AS final_record_count 
FROM complete_joint_dataset

UNION

SELECT 
    'left join' AS join_type,
    COUNT(*) AS final_record_count 
FROM complete_joint_dataset_left

NOTICE: table "complete_joint_dataset" does not exist, skipping

Commands completed successfully

Commands completed successfully

NOTICE: table "complete_joint_dataset_left" does not exist, skipping

Commands completed successfully

Commands completed successfully

(2 row(s) affected)

Total execution time: 00:00:00.178

join_type,final_record_count
inner join,16044
left join,16044
